# pegasus-x-large  demo

- for [this fine-tuned version](https://huggingface.co/pszemraj/pegasus-x-large-book-summary) of pegasus-x by Google


---


In [ ]:
#@markdown add auto-Colab formatting with `IPython.display`
from IPython.display import HTML, display
# colab formatting
def set_css():
    display(
        HTML(
            """
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  """
        )
    )

get_ipython().events.register("pre_run_cell", set_css)

## setup


In [ ]:
!pip install transformers accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.2 MB/s eta 0:00:00


In [ ]:
!nvidia-smi

Fri Jun  9 01:45:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    43W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
hf_tag = "pszemraj/pegasus-x-large-book-summary" #@param {type:"string"}


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(hf_tag)

model = AutoModelForSeq2SeqLM.from_pretrained(
    hf_tag,
)


In [ ]:
import torch
from transformers import pipeline

summarizer = pipeline('summarization',
                      model=model,
                      tokenizer=tokenizer,
                      device= 0 if torch.cuda.is_available() else -1,
                      )

In [ ]:
import pandas as pd

# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## load input text

## summary

In [ ]:
#@markdown define generation parameters
#@markdown - for even better performance increase `num_beams`
params = {
    "max_length": 256,
    "min_length": 8,
    "no_repeat_ngram_size": 3,
    "early_stopping": True,
    "repetition_penalty": 3.5,
    "length_penalty": 0.2,
    "encoder_no_repeat_ngram_size": 3,
    "num_beams": 4,
}


In [ ]:
import pandas as pd
import re

# Load the Excel file
input_file = "/content/drive/MyDrive/All_Patent.xlsx"
data_frame = pd.read_excel(input_file)

# Initialize an empty list to store the summaries
summaries = []

# Loop through the rows
for index, row in data_frame.iterrows():
    # Get the abstract and claims
    abstract = row['Abstract']
    claims = row['Claims']

    # Clean the abstract and claims text
    abstract = re.sub(r'[^\x00-\x7F]+', '', abstract)
    claims = re.sub(r'[^\x00-\x7F]+', '', claims)

    # Combine the abstract and claims
    input_text = abstract + ' ' + claims

    # Encode the combined text to handle special characters
    input_text = input_text.encode('unicode_escape').decode()

    # Generate the summary
    summary = summarizer(input_text, **params)[0]['summary_text']

    # Add the summary to the list
    summaries.append(summary)

# Add the summaries to the data frame
data_frame['Summary'] = summaries

# Save the data frame to a new Excel file
output_file = '/content/drive/MyDrive/pegasus-x-large-booksum-1.xlsx'
data_frame.to_excel(output_file, index=False)

Token indices sequence length is longer than the specified maximum sequence length for this model (1726 > 1024). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
%%time
result = summarizer(long_text, **params)
print(f"summarized inputs are:\n\n{result[0]['summary_text']}")


summarized inputs are:

A method of encoding a viewport, which is a portion of a video stream, comprises selecting a first viewport and encoding the first viewport with a higher quality than a second viewport. The second viewport is then encoded with a lower quality so as to provide a gradual change in the quality of the video between the two viewports.
CPU times: user 1.8 s, sys: 119 ms, total: 1.92 s
Wall time: 3.92 s
